In [1]:
import wallycore as wally

In [2]:
PREFIXES = {
        "MAINNET" : {"P2PKH": 0, "P2SH" : 5, "BECH32" : "bc"},
        "TESTNET" : {"P2PKH": 111, "P2SH" : 196, "BECH32" : "tb"}
}

In [3]:
def find_address_type(address_string):
    for network in PREFIXES:
        if PREFIXES[network]["BECH32"] == address_string[:2].lower() :
            return (network, "BECH32")

    try:
        address_bytes = wally.base58_to_bytes(address_string, wally.BASE58_FLAG_CHECKSUM)
        if len(address_bytes) != 21:
            raise Exception("Invalid length")
        first_byte = address_bytes[0]
        for network in PREFIXES:
            for address_type in PREFIXES[network]:
                if PREFIXES[network][address_type] == first_byte:
                    return (network, address_type)
    except:
        pass
    
    raise Exception("Invalid address")

In [4]:
def extract_data_bech32(address_string, network):
    return wally.addr_segwit_to_bytes(address_string, PREFIXES[network]["BECH32"], 0)

def extract_data(address_string): 
    address_bytes = wally.base58_to_bytes(address_string, wally.BASE58_FLAG_CHECKSUM)
    return address_bytes[1:]


In [5]:
def get_script_pub_key(address_string):
    (network, address_type) = find_address_type(address_string)
    if address_type == "BECH32":
        return (extract_data_bech32(address_string, network), network)
    data = extract_data(address_string)
    
    if address_type == "P2PKH":
        #<ScriptPubKey=OP_DUP OP_HASH160<Public KeyHash> OP_EQUAL OP_CHECKSIG
        return (bytearray([wally.OP_DUP, wally.OP_HASH160, 20]) + data + bytearray([wally.OP_EQUAL, wally.OP_CHECKSIG]), network) 
        
    if address_type == "P2SH":
        #<ScriptPubKey=OP_HASH160<ScriptHash> OP_EQUAL
        return (bytearray([wally.OP_HASH160, 20]) + data + bytearray([wally.OP_EQUAL]), network)


In [6]:
testcases = ["BC1QW508D6QEJXTDG4Y5R3ZARVARY0C5XW7KV8F3T4", 
             "tb1qnf2g836zetxxzj6wzqgneauwp2xynzl56e88gl06938q085wc5eqynu2h6",
             "2N16oE62ZjAPup985dFBQYAuy5zpDraH7Hk",
             "35c5G57dKKaQDpkR8FfyCADuJsssoHgdnH",
             "1H4R6GA8S23uuQSeBi8ySLm965g2eCD8Za"]
for address in testcases:
    (script, network) = get_script_pub_key(address)
    print(address, "->", wally.hex_from_bytes(script), network)

BC1QW508D6QEJXTDG4Y5R3ZARVARY0C5XW7KV8F3T4 -> 0014751e76e8199196d454941c45d1b3a323f1433bd6 MAINNET
tb1qnf2g836zetxxzj6wzqgneauwp2xynzl56e88gl06938q085wc5eqynu2h6 -> 00209a5483c742cacc614b4e10113cf78e0a8c498bf4d64e747dfa2c4e079e8ec532 TESTNET
2N16oE62ZjAPup985dFBQYAuy5zpDraH7Hk -> a9145629021f7668d4ec310ac5e99701a6d6cf95eb8f87 TESTNET
35c5G57dKKaQDpkR8FfyCADuJsssoHgdnH -> a9142af1356a5fd14e96443c8039ef2d6d0679face5d87 MAINNET
1H4R6GA8S23uuQSeBi8ySLm965g2eCD8Za -> 76a914b0275ece9a509c048f79a13f8de88ad19a8d538b87ac MAINNET
